## Coursera Capstone Project

In [180]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

# Matplotlib and associated plotting modules
# import matplotlib.cm as cm
# import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import wget
import requests
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')


Libraries imported.


## 1. Download the zip code list of Washington DC and load in a dataframe

In [58]:
# Download the zip code list of Washington DC using opendatasoft API into a file
url="https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=Washington+DC&rows=300&refine.state=DC"
response = requests.get(url)
jsonDataStr = json.dumps(response.json())
# with open('washington_dc_zip.json', 'w') as json_file:
#   json.dump(jsonDataStr, json_file)
# open('washington_dc_zip.json', 'w').write(jsonDataStr)

113926

In [181]:
# Load zip code JSON to Dataframe
with open("washington_dc_zip.json") as f:
  jsonData = json.load(f)

zipcodesDf = pd.json_normalize(jsonData["records"])
zipcodesDf = zipcodesDf.drop("datasetid", axis=1).drop("recordid", axis=1).drop("record_timestamp", axis=1)
zipcodesDf = zipcodesDf.drop("fields.geopoint", axis=1).drop("geometry.coordinates", axis=1).drop("fields.state", axis=1).drop("fields.dst", axis=1)
zipcodesDf = zipcodesDf.drop("fields.timezone", axis=1).drop("geometry.type", axis=1)
zipcodesDf.rename(columns={"fields.city": "city", "fields.zip": "zip", "fields.longitude": "long", "fields.latitude": "lat"}, inplace=True)
zipcodesDf.head()

,city,zip,long,lat
0,Washington,20210,-77.014647,38.893311
1,Washington,20202,-77.014647,38.893311
2,Washington,20537,-77.025133,38.894097
3,Washington,20068,-77.014647,38.893311
4,Washington,20076,-77.014647,38.893311


In [184]:
# If multiple zip codes have same lat, long, then keep only the last one
zipcodesDf.drop_duplicates(subset=["long", "lat"], keep="last", inplace=True)

In [185]:
print("Shape of the data is {}".format(zipcodesDf.shape))

Shape of the data is (98, 4)


## Using Foursquare API, get the recommended venue list for each zip code / lat long from

In [92]:
## hidden
CLIENT_ID = "YIOJTSFKMWE35RORHMBN0AMLFEEGDU4C4KGH3M3VHDRTL43L"
CLIENT_SECRET = "FVH0DHE5M11DHG2TV2XMVGOL2OKAOMXW1DEATV33RAZKVSLH"
VERSION = '20180605' # Foursquare API version
LIMIT=50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YIOJTSFKMWE35RORHMBN0AMLFEEGDU4C4KGH3M3VHDRTL43L
CLIENT_SECRET:FVH0DHE5M11DHG2TV2XMVGOL2OKAOMXW1DEATV33RAZKVSLH


In [76]:
# FOUR_SQUARE_EXPLORE_URL = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v=20180602&limit={}".format(CLIENT_ID, CLIENT_SECRET, LIMIT)

In [93]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zip', 
                  'zip latitude', 
                  'zip longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [186]:
# Read from Foursquare and write this data to CSV for later reload so that we don't run into quota limits of FourSquare APIs
# venuesDf = getNearbyVenues(zipcodesDf["zip"], zipcodesDf["lat"], zipcodesDf["long"])
# venuesDf.to_csv("foursquare_data.csv")

# venuesDf = pd.read_csv("foursquare_data.csv")
venuesDf.head()

,zip,zip latitude,zip longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20065,38.883412,-77.028198,ARTECHOUSE,38.884299,-77.029266,Art Gallery
1,20065,38.883412,-77.028198,"Mandarin Oriental, Washington DC",38.883659,-77.030319,Hotel
2,20065,38.883412,-77.028198,International Spy Museum,38.883895,-77.025539,Museum
3,20065,38.883412,-77.028198,Maine Avenue Fish Market,38.881145,-77.028118,Fish Market
4,20065,38.883412,-77.028198,Falafel Inc,38.881443,-77.027500,Falafel Restaurant


In [98]:
# venuesDf.groupby('zip').count()

In [187]:
print('There are {} unique categories.'.format(len(venuesDf['Venue Category'].unique())))
print('Shape is {} '.format(venuesDf.shape))
venuesDf.head()

There are 278 unique categories.
Shape is (3776, 7) 


,zip,zip latitude,zip longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,20065,38.883412,-77.028198,ARTECHOUSE,38.884299,-77.029266,Art Gallery
1,20065,38.883412,-77.028198,"Mandarin Oriental, Washington DC",38.883659,-77.030319,Hotel
2,20065,38.883412,-77.028198,International Spy Museum,38.883895,-77.025539,Museum
3,20065,38.883412,-77.028198,Maine Avenue Fish Market,38.881145,-77.028118,Fish Market
4,20065,38.883412,-77.028198,Falafel Inc,38.881443,-77.027500,Falafel Restaurant


## Analyze each zip code

In [188]:
# one hot encoding
dc_onehot = pd.get_dummies(venuesDf[['Venue Category']], prefix="", prefix_sep="")

# add zip column back to dataframe
dc_onehot['zip'] = venuesDf['zip'] 

# # move zip column to the first column
fixed_columns = [dc_onehot.columns[-1]] + list(dc_onehot.columns[:-1])
dc_onehot = dc_onehot[fixed_columns]

dc_onehot.head()

,zip,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Check Cashing Service,Chinese Restaurant,Christmas Market,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wi

In [189]:
print('Shape is {} '.format(dc_onehot.shape))

Shape is (3776, 279) 


### Next, let's group rows by zip code and by taking the mean of the frequency of occurrence of each category

In [190]:
dc_grouped = dc_onehot.groupby('zip').mean().reset_index()
dc_grouped.head()

,zip,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Check Cashing Service,Chinese Restaurant,Christmas Market,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Synagogue,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wi

In [191]:
print('Shape is {} '.format(dc_grouped.shape))

Shape is (98, 279) 


In [203]:
# Create a new dataframe with "Restaurant" columns
cols = [col for col in dc_onehot.columns]
cols = [col for col in dc_onehot.columns if col.find("Restaurant") >= 0]
cols.insert(0, "zip")
cols

restaurantsDf = dc_onehot[cols]
restaurantsDf.head()

,zip,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Cuban Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,20065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,20065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,20065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [173]:
# temp = dc_grouped[dc_grouped['zip'] == "20001"].T.reset_index()
# temp.columns = ['venue','freq']
# temp = temp.iloc[1:]
# temp['freq'] = temp['freq'].astype(float)
# temp = temp.round({'freq': 2})
# print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

In [128]:
num_top_venues = 5

for zipcode in dc_grouped['zip']:
    print("----zipcode="+ zipcode +"----")
    temp = dc_grouped[dc_grouped['zip'] == zipcode].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----zipcode=20001----
                             venue  freq
0                    Grocery Store  0.14
1                      Coffee Shop  0.07
2  Southern / Soul Food Restaurant  0.07
3             Gym / Fitness Center  0.07
4                  Thai Restaurant  0.07


----zipcode=20002----
              venue  freq
0               Bar  0.08
1       Pizza Place  0.06
2        Taco Place  0.04
3  Asian Restaurant  0.04
4          Wine Bar  0.04


----zipcode=20003----
                venue  freq
0    Sushi Restaurant  0.04
1              Bakery  0.04
2         Coffee Shop  0.04
3  Italian Restaurant  0.04
4                 Spa  0.04


----zipcode=20004----
         venue  freq
0        Hotel  0.06
1   Art Museum  0.04
2  Coffee Shop  0.04
3  Salad Place  0.04
4      Theater  0.04


----zipcode=20005----
                           venue  freq
0                          Hotel  0.12
1                    Coffee Shop  0.10
2             Salon / Barbershop  0.06
3                     Food Tru